# BASIC TESTS

In [14]:
#enchant dict

In [12]:
#stopwords
from nltk.corpus import stopwords 
stop = set(stopwords.words('english'))

text = "i am produced by a producer"
text = [i for i in text.lower().split() if i not in stop]
text

['produced', 'producer']

In [2]:
#synonyms
#from nltk.corpus import wordnet

synonyms = []
for syn in wordnet.synsets("fool"):
    for l in syn.lemmas():
        synonyms.append(l.name())
print(set(synonyms))


{'shoot', 'dissipate', 'dupe', 'frivol_away', 'chump', 'mark', 'gull', 'tomfool', 'cod', 'slang', 'put_on', 'arse_around', 'patsy', 'fool', 'soft_touch', 'horse_around', 'sucker', 'fool_around', 'put_one_over', 'muggins', 'sap', 'mug', 'take_in', 'motley_fool', 'jester', 'befool', 'fool_away', 'fritter', 'saphead', 'fritter_away', 'put_one_across', 'fall_guy'}


In [84]:
#POS tagging N/V
#import spacy
#nlp = spacy.load("en_core_web_lg")

text = ("the dumb Jack")
doc = nlp(text)
# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])
print("Adj:", [token.lemma_ for token in doc if token.pos_ == "ADJ"])

Noun phrases: ['the dumb Jack']
Verbs: []
Adj: ['dumb']


In [6]:
#Active / Passive
from modules.functions import is_active

print(is_active("He is cutting wood with the saw"))
print(is_active("the wood is being cut by him with a saw"))

1
0


In [264]:
sent2

"the 3 programs are being run by Jake's amazing computer wonderfully"

# COMPARE AND REPLACE

In [131]:
import spacy
nlp = spacy.load("en_core_web_lg")
from nltk.corpus import wordnet
from num2words import num2words
from modules.functions import is_active
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

ModuleNotFoundError: No module named 'modules.functions'

In [143]:
#Gets sentence returns obj with POS tagged
def tagSentence(sentence):
    doc = nlp(sentence)
    obj = {}
    obj["NOUN"] = [token.lemma_.lower() for token in doc if token.pos_ == "NOUN"]
    obj["PROPN"] = [token.lemma_.lower() for token in doc if token.pos_ == "PROPN"]
    obj["PRON"] = [token.lemma_.lower() for token in doc if token.pos_ == "PRON"]
    obj["VERB"] = [token.lemma_.lower() for token in doc if token.pos_ == "VERB" and token.lemma_.lower() != "be"]
    obj["ADJ"] = [token.lemma_.lower() for token in doc if token.pos_ == "ADJ"]
    obj["ADV"] = [token.lemma_.lower() for token in doc if token.pos_ == "ADV"]
    obj["NUM"] = [token.lemma_.lower() for token in doc if token.pos_ == "NUM"]
    obj["SYM"] = [token.lemma_.lower() for token in doc if token.pos_ == "SYM"]
    obj["X"] = [token.lemma_.lower() for token in doc if token.pos_ == "X"]
    return obj

#Only for sent2
def reTagSentence(sentence2):
    obj = {}
    doc = nlp(sentence2)
    obj["NOUN"] = [token.lemma_.lower() for token in doc if token.pos_ == "NOUN"]
    obj["PROPN"] = [token.lemma_.lower() for token in doc if token.pos_ == "PROPN"]
    obj["PRON"] = [token.lemma_.lower() for token in doc if token.pos_ == "PRON"]
    obj["ADJ"] = [token.lemma_.lower() for token in doc if token.pos_ == "ADJ"]
    obj["VERB"] = [token.lemma_.lower() for token in doc if token.pos_ == "VERB" and token.lemma_.lower() != "be"]
    return obj

#Gets obj with POS tagged, returns obj with POS tagged + synonyms(only for obj2)
def findSynonyms(obj):
    #NOUN SYNONYMS FOUND IN COMPARE() ITSELF DUE TO NOUN ORDER CHECK
    #ADJ SYNONYMS FOUND IN COMPARE() ITSELF DUE TO ADJ ORDER CHECK
    #VERB
    if(len(obj["VERB"])>0):
        tempArr = []
        for word in obj["VERB"]:
            for syn in wordnet.synsets(word):
                for l in syn.lemmas():
                    tempArr.append(l.name())
        obj["VERB"] = list(set(obj["VERB"]+tempArr))
    
    #ADV   
    if(len(obj["ADV"])>0):
        tempArr = []
        for word in obj["ADV"]:
            for syn in wordnet.synsets(word):
                for l in syn.lemmas():
                    tempArr.append(l.name())
        obj["ADV"] = list(set(obj["ADV"]+tempArr))
    #X  
    if(len(obj["X"])>0):
        tempArr = []
        for word in obj["X"]:
            for syn in wordnet.synsets(word):
                for l in syn.lemmas():
                    tempArr.append(l.name())
        obj["X"] = list(set(obj["X"]+tempArr))
    return obj
     
#Comparison
def compare(obj1,obj2,sentence2):
    #original object 2
    obj2a = reTagSentence(sentence2)
    result = {}
    #NOUN, PROPN, PRON, VERB, ADJ, ADV, NUM, SYM, X - CRITICAL
    #NOUN
    if(len(obj1["NOUN"])>0):
        result["NOUN"] = 0
        status = 0
        for i in range(len(obj1["NOUN"])): 
            word1 = obj1["NOUN"][i]
            for j in range(len(obj2["NOUN"])): 
                word2 = obj2["NOUN"][j]
                tempArr = []
                for syn in wordnet.synsets(word2):
                    for l in syn.lemmas():
                        tempArr.append(l.name())
                for tempWord in tempArr:
                    if(word1 == tempWord):
                        status = 1
                        sentence2 = sentence2.replace(word2,word1)
                        result["NOUN"] += 1
                        break
            if(status == 1):
                status = 0
            else:
                break
    
    #PROPN      
    if(len(obj1["PROPN"])>0):
        result["PROPN"] = 0
        for i in range(len(obj1["PROPN"])): 
            word1 = obj1["PROPN"][i]
            for word2 in obj2["PROPN"]:
                if(word1 == word2):
                    result["PROPN"] += 1
    #PRON               
    if(len(obj1["PRON"])>0):
        result["PRON"] = 0
        for word1 in obj1["PRON"]:
            for word2 in obj2["PRON"]:
                if(word1 == word2):
                    result["PRON"] += 1
    #VERB
    if(len(obj1["VERB"])>0):
        result["VERB"] = 0
        for word1 in obj1["VERB"]:
            for word2 in obj2["VERB"]:
                if(word1 == word2):
                    result["VERB"] += 1 
    #ADJ 
    if(len(obj1["ADJ"])>0):
        result["ADJ"] = 0
        status = 0
        for i in range(len(obj1["ADJ"])): 
            word1 = obj1["ADJ"][i]
            for j in range(len(obj2["ADJ"])): 
                word2 = obj2["ADJ"][j]
                tempArr = []
                for syn in wordnet.synsets(word2):
                    for l in syn.lemmas():
                        tempArr.append(l.name())
                for tempWord in tempArr:
                    if(word1 == tempWord):
                        status = 1
                        sentence2 = sentence2.replace(word2,word1)
                        result["ADJ"] += 1
                        break

    #ADV
    if(len(obj1["ADV"])>0):
        result["ADV"] = 0
        for word1 in obj1["ADV"]:
            for word2 in obj2["ADV"]:
                if(word1 == word2):
                    result["ADV"] += 1 
    #NUM     
    if(len(obj1["NUM"])>0):
        #NUM2WORD
        for i in range(len(obj1["NUM"])):
            if(obj1["NUM"][i].isdigit()):
                obj1["NUM"][i] = num2words(obj1["NUM"][i])
        for i in range(len(obj2["NUM"])):
            if(obj2["NUM"][i].isdigit()):
                obj2["NUM"][i] = num2words(obj2["NUM"][i])
        #NORMAL
        result["NUM"] = 0
        for word1 in obj1["NUM"]:
            for word2 in obj2["NUM"]:
                if(word1 == word2):
                    result["NUM"] += 1
    #SYM
    if(len(obj1["SYM"])>0):
        result["SYM"] = 0
        for word1 in obj1["SYM"]:
            for word2 in obj2["SYM"]:
                if(word1 == word2):
                    result["SYM"] += 1
    #X-OTHERS
    if(len(obj1["X"])>0):
        result["X"] = 0
        for word1 in obj1["X"]:
            for word2 in obj2["X"]:
                if(word1 == word2):
                    result["X"] += 1
    tempObj = {
        'NOUN':len(obj1['NOUN']),
        'PROPN':len(obj1['PROPN']),
        'PRON':len(obj1['PRON']),
        'VERB':len(obj1['VERB']),
        'ADJ':len(obj1['ADJ']),
        'ADV':len(obj1['ADV']),
        'SYM':len(obj1['SYM']),
        'NUM':len(obj1['NUM']),
        'X':len(obj1['X'])
    }
    res = {
        'teachArr':tempObj,
        'studArr':result
    }
    #TUPLE UNPACK AND GET BOTH VALUES
    return res,sentence2

#Is active
def is_active(sentence):
    doc = nlp(sentence)
    passive_rule = [{'DEP': 'nsubjpass'}, {'DEP': 'aux', 'OP': '*'}, {'DEP': 'auxpass'}, {'TAG': 'VBN'}]
    matcher.add('Passive', None, passive_rule)
    matches = matcher(doc)
    if matches:
        return False
    else:
        return True

In [144]:
sent1 = "stupid Jake's awesome calculator is executing the three programs marvellously"
sent2 = "the 3 programs are being run by Jake's amazing computer wonderfully"
object1 = tagSentence(sent1)
object2 = tagSentence(sent2)
print(object1)
print(object2)
#OBJECT1 #TEACHER OBJECT NOT TO BE SYNONYMIZED - TO CHECK IF EVERYTHING IS PRESENT
object2 = findSynonyms(object2)

{'NOUN': ['calculator', 'program'], 'PROPN': ['jake'], 'PRON': [], 'VERB': ['execute'], 'ADJ': ['stupid', 'awesome'], 'ADV': ['marvellously'], 'NUM': ['three'], 'SYM': [], 'X': []}
{'NOUN': ['program', 'computer'], 'PROPN': ['jake'], 'PRON': [], 'VERB': ['run'], 'ADJ': ['amazing'], 'ADV': ['wonderfully'], 'NUM': ['3'], 'SYM': [], 'X': []}


In [145]:
print(sent1)
print(sent2)
res,sent2 = compare(object1,object2,sent2)
print(res)
print(sent1)
print(sent2)

stupid Jake's awesome calculator is executing the three programs marvellously
the 3 programs are being run by Jake's amazing computer wonderfully
{'teachArr': {'NOUN': 2, 'PROPN': 1, 'PRON': 0, 'VERB': 1, 'ADJ': 2, 'ADV': 1, 'SYM': 0, 'NUM': 1, 'X': 0}, 'studArr': {'NOUN': 2, 'PROPN': 1, 'VERB': 1, 'ADJ': 1, 'ADV': 1, 'NUM': 1}}
stupid Jake's awesome calculator is executing the three programs marvellously
the 3 programs are being run by Jake's awesome calculator wonderfully


In [23]:
#end of compare and replace

## NOUN/ADJECTIVE ORDER CHECK

In [139]:
#These sentences will be with replaced words for sent2 alone
#For NOUN check
def NCsplitOnVerb(sentence):
    sent = {}
    doc = nlp(sentence)
    sent["TYPE"] = is_active(sentence)
    sent["NPV"] = [token.lemma_.lower() for token in doc if token.pos_=="NOUN" 
     or token.pos_=="PRON" 
     or token.pos_=="PROPN"
     or token.pos_=="VERB" and token.lemma_.lower()!="be"]
    sent["NPV"] = ' '.join(sent["NPV"])
    sent["V"] = [token.lemma_.lower() for token in doc if token.pos_=="VERB" and token.lemma_.lower()!="be"]
    sent["NP"]= sent["NPV"].split(sent["V"][0])
    sent["NP"] = [word.replace(' ','') for word in sent["NP"]]
    sent["NP"] = [''.join(sorted(word)) for word in sent["NP"]]
    return sent

#Check order of appearance of a noun
def checkOrderOfNoun(sent1,sent2):
    sent1obj = NCsplitOnVerb(sent1)
    sent2obj = NCsplitOnVerb(sent2)
    index1 = []
    index2 = []
    for i in range(len(sent1obj["NP"])):
        index1.append(i)
        try:
            index2.append(sent2obj["NP"].index(sent1obj["NP"][i]))
        except:
            return False
    if(sent1obj["TYPE"]==sent2obj["TYPE"]):
        #len(index1) = len(index2)
        for i in range(len(index1)):
            if(index1[i]!=index2[i]):
                return False
        return True
    elif(sent1obj["TYPE"]!=sent2obj["TYPE"]):
        for i in range(len(index1)):
            if(index1[i]==index2[i]):
                return False
        return True

In [162]:
#For ADJ check
def ACsplitOnVerb(sentence):
    sent = {}
    doc = nlp(sentence)
    sent["TYPE"] = is_active(sentence)
    sent["NPVA"] = [token.lemma_.lower() for token in doc if token.pos_=="NOUN" 
     or token.pos_=="PRON" 
     or token.pos_=="PROPN"
     or token.pos_=="ADJ"
     or token.pos_=="VERB" and token.lemma_.lower()!="be"]
    sent["NPVA"] = ' '.join(sent["NPVA"])
    sent["V"] = [token.lemma_.lower() for token in doc if token.pos_=="VERB" and token.lemma_.lower()!="be"]
    sent["NPA"]= sent["NPVA"].split(sent["V"][0])
    sent["NPA"] = [word.replace(' ','') for word in sent["NPA"]]
    sent["NPA"] = [''.join(sorted(word)) for word in sent["NPA"]]
    return sent

#Check order of appearance of an adjective
def checkOrderOfAdjective(sent1,sent2):
    sent1obj = ACsplitOnVerb(sent1)
    sent2obj = ACsplitOnVerb(sent2)
    index1 = []
    index2 = []
    for i in range(len(sent1obj["NPA"])):
        index1.append(i)
        try:
            index2.append(sent2obj["NPA"].index(sent1obj["NPA"][i]))
        except:
            return False
    if(sent1obj["TYPE"]==sent2obj["TYPE"]):
        #len(index1) = len(index2)
        for i in range(len(index1)):
            if(index1[i]!=index2[i]):
                return False
        return True
    elif(sent1obj["TYPE"]!=sent2obj["TYPE"]):
        for i in range(len(index1)):
            if(index1[i]==index2[i]):
                return False
        return True

In [163]:
checkOrderOfNoun(sent1,sent2)

True

In [164]:
checkOrderOfNoun("Jack shot Bill","Bill was shot by Jack")

True

In [165]:
checkOrderOfAdjective(sent1,sent2)

False

In [166]:
checkOrderOfAdjective("hot Jack shot Bill","Bill was shot by hot Jack")

True

In [1]:
###END###

## SIMILARITY CHECK

In [125]:
obj = {
    'teachMark' :{'NOUN': 2, 'PROPN': 1, 'PRON':0, 'VERB': 1, 'ADJ': 2, 'ADV': 1, 'SYM':0 ,'NUM': 1},
    'studMark' :{'NOUN': 2, 'PROPN': 1, 'VERB': 1, 'ADJ': 0, 'ADV': 0, 'NUM': 1}
}

In [126]:
def simCheck(obj, studSum=0 ,teachSum=0): 
    FACT = {'NOUN': 1, 'PROPN': 1, 'PRON':1, 'VERB': 1, 'ADJ': 0.3, 'ADV': 0.3, 'SYM':1 ,'NUM': 1}
    teachMark = obj['teachMark']
    studMark = obj['studMark']
    for key in teachMark:
        if(teachMark[key]!=0):
            try:
                studMark[key]
                totalval = teachMark[key]
                val = ((teachMark[key]-studMark[key])*FACT[key])+studMark[key]
                studSum+=val
                teachSum+=totalval
            except:
                continue
    return studSum/teachSum

In [127]:
simCheck(obj)

0.7374999999999999

# CHUNK -> COMPARE

In [1]:
from modules.compare import compare, is_active, checkOrderOfNoun, checkOrderOfAdjective
from modules.metrics import simCheck

In [112]:
#convert teach array to this format
teach = [
        "The dog is chasing the fat cat",
        "The cat is eating the mouse",
        "The mouse is eating food"
]

student = [
    "The mouse is laughing",
    "The cat is being chased by the dog",
    "the mouse is being eaten by the cat"
]

In [120]:
#teach and student are array of sentences
#AFTER CHUNKING

######## ADJUST SIM SCORE MULTIPLICATION
def compareMain(teach,student):
    arr=[]
    for i in range(len(teach)):
        obj={}
        obj['s']=teach[i]
        obj['c']=0
        arr.append(obj)
    teach = arr
    for teachObj in teach:
        for sent in student:
            res , sent2 = compare(teachObj['s'],sent)
            simScore = simCheck(res)
            if(res['teachMark']['VERB']>0):
                if(checkOrderOfNoun(teachObj['s'],sent2)==False):
                    simScore*=0
                if(checkOrderOfAdjective(teachObj['s'],sent2)==False):
                    simScore*=0.95              #################
            if(simScore>0.4):                   #################
                teachObj['SIM'] = simScore
                teachObj['c']=1
    #teach is an object with attribute 's' , 'c' and 'SIM' if matched(result)
    return teach

In [123]:
res = compareMain(teach,student)

In [3]:
res

NameError: name 'res' is not defined